# **Notebook 03 — Feature Engineering: Pixel Drilling and Dataset Construction**

## Predictive Archaeological Modelling — Peru (v2)
**Author:** Yishar Piero Nieto Barrientos

---

### Objective

Extract pixel values from **all raster layers** registered in `src/config.py` at every sample coordinate generated in Notebook 02, producing the unified training dataset.

> **Scalability:** The variable list is read from `src/config.py`.
> Adding a new raster to the config propagates here automatically.

### Pipeline

| Step | Description | Output |
|------|-------------|--------|
| 1 | Load presence/absence samples from Notebook 02 | `presencias.csv` + `ausencias.csv` |
| 2 | Register all raster variables (auto from config) | Raster inventory |
| 3 | Pixel drilling: extract values at all sample coordinates | Feature matrix |
| 4 | NoData cleaning and export | `dataset_entrenamiento.csv` |

**Depends on:** Notebook 01 (validation passed), Notebook 02 (samples generated).
**Next:** Notebook 04 (Training and Model Validation).

### **1. Environment Setup**

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm

from src.config import (
    SAMPLES_DIR, FEATURES_DIR,
    figures_dir, get_rasters, get_raster_paths, get_feature_names,
)

FIGURES_DIR = figures_dir("03_feature_eng")

print("Environment configured.")

Environment configured.


### **2. Raster Variable Registry (from config)**

All predictor layers are read from `src/config.py`.  Each entry maps a column name to a raster file path.

In [3]:
# ── Variable registry (auto from config) ──────────────────────────────────────
rasters = get_raster_paths()
all_vars = get_rasters()

print(f"{'Variable':<22} {'Version':>8} {'Type':>12} {'Status':>8}")
print("─" * 54)
for rv in all_vars:
    status = "OK" if rv.exists else "MISSING"
    print(f"{rv.name:<22} {rv.version:>8} {rv.var_type.value:>12} {status:>8}")

n_v1 = len([r for r in all_vars if r.version == "v1"])
n_v2 = len([r for r in all_vars if r.version == "v2"])
print(f"\nTotal: {len(all_vars)} variables ({n_v1} v1 + {n_v2} v2)")

Variable                Version         Type   Status
──────────────────────────────────────────────────────
pendiente                    v1   continuous       OK
rugosidad                    v1   continuous       OK
dist_rios                    v1     distance       OK
dist_lagos                   v1     distance       OK
dist_qhapaq                  v1     distance       OK
dist_declarados              v1     distance       OK
dist_g1                      v1     distance       OK
dist_g2                      v1     distance       OK
dist_g3                      v1     distance       OK
altitud                      v2   continuous       OK
aspecto                      v2   continuous       OK
curvatura                    v2   continuous       OK
tpi                          v2   continuous       OK
twi                          v2   continuous       OK
pisos_ecologicos             v2  categorical       OK

Total: 15 variables (9 v1 + 6 v2)


### **3. Feature Extraction (Pixel Drilling)**

For each sample coordinate, the corresponding pixel value is extracted from every raster layer. Rows with NoData values (out-of-bounds or edge artefacts) are dropped. The resulting tabular dataset is exported to `data/features/dataset_entrenamiento.csv`.

In [4]:
print("--- Pixel Drilling ---")

# ==============================================================================
# 1. LOAD AND MERGE SAMPLES
# ==============================================================================
ruta_si = os.path.join(SAMPLES_DIR, "presencias.csv")
ruta_no = os.path.join(SAMPLES_DIR, "ausencias.csv")

if not (os.path.exists(ruta_si) and os.path.exists(ruta_no)):
    raise FileNotFoundError("Sample CSV files not found. Run Notebook 02 first.")

df_si = pd.read_csv(ruta_si)
df_no = pd.read_csv(ruta_no)
df_master = pd.concat([df_si, df_no], ignore_index=True)

print(f"  Total samples:  {len(df_master)}")
print(f"  Presences (1):  {len(df_si)}")
print(f"  Absences (0):   {len(df_no)}")

# ==============================================================================
# 2. EXTRACTION (auto from config registry)
# ==============================================================================
print("\n  Extracting pixel values ...")
coords = list(zip(df_master["X_102033"], df_master["Y_102033"]))

for var_name, raster_path in tqdm(rasters.items(), desc="  Variables"):
    if os.path.exists(raster_path):
        with rasterio.open(raster_path) as src:
            values = [x[0] for x in src.sample(coords)]
            df_master[var_name] = values
            if src.nodata is not None:
                df_master.loc[df_master[var_name] == src.nodata, var_name] = np.nan
        print(f"  [ok] {var_name}")
    else:
        df_master[var_name] = np.nan
        print(f"  [MISSING] {var_name}")

# ==============================================================================
# 3. CLEAN AND SAVE
# ==============================================================================
n_before = len(df_master)
df_final = df_master.dropna()
n_after = len(df_final)

print(f"\n  Rows dropped (NoData / out of bounds): {n_before - n_after}")
print(f"  Final dataset: {n_after} observations x {len(rasters)} features")

ruta_salida = os.path.join(FEATURES_DIR, "dataset_entrenamiento.csv")
df_final.to_csv(ruta_salida, index=False)

print(f"  Saved: {ruta_salida}")
print(f"\n  Preview:")
df_final.head()

--- Pixel Drilling ---
  Total samples:  29572
  Presences (1):  14786
  Absences (0):   14786

  Extracting pixel values ...


  Variables:   0%|          | 0/15 [00:00<?, ?it/s]/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype=np.int32)
/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype=np.int32)
  Variables:   7%|▋         | 1/15 [01:08<15:56, 68.31s/it]

  [ok] pendiente


/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype=np.int32)
/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype=np.int32)
  Variables:  13%|█▎        | 2/15 [02:16<14:43, 67.95s/it]

  [ok] rugosidad


...

  [ok] twi


/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype=np.int32)
/home/mazapan/Documentos/arqueologia-predictiva/venv/lib/python3.14/site-packages/rasterio/transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype=np.int32)
  Variables: 100%|██████████| 15/15 [12:53<00:00, 51.56s/it]

  [ok] pisos_ecologicos

  Rows dropped (NoData / out of bounds): 2191
  Final dataset: 27381 observations x 15 features


  Saved: /home/mazapan/Documentos/arqueologia-predictiva/data/features/dataset_entrenamiento.csv

  Preview:


,X_102033,Y_102033,Origen,Target,pendiente,rugosidad,dist_rios,dist_lagos,dist_qhapaq,dist_declarados,dist_g1,dist_g2,dist_g3,altitud,aspecto,curvatura,tpi,twi,pisos_ecologicos
0,-1.794915e+06,2.428515e+06,Sitio_Declarado,1,2.569503,3.464102,30.000000,2731.483154,30.000000,42.426407,1955.760742,0.000000,35445.734375,3802.0,291.801392,-0.002222,-3.196203,6.505041,4.0
1,-1.780363e+06,2.414741e+06,Sitio_Declarado,1,4.431820,5.830952,954.829834,60.000000,1950.230713,0.000000,2640.000000,1890.952148,37885.343750,3855.0,36.253838,-0.001111,-7.797468,5.958617,4.0
2,-1.778420e+06,2.414945e+06,Sitio_Declarado,1,4.727942,6.708204,84.852814,540.832642,0.000000,30.000000,630.000000,0.000000,37040.683594,3862.0,139.085617,-0.002222,-5.420886,5.893661,4.0
3,-1.791737e+06,2.426272e+06,Sitio_Declarado,1,3.372287,12.124355,90.000000,2508.007324,30.000000,0.000000,2502.258789,0.000000,34444.000000,3824.0,351.869904,0.012222,-41.791138,6.232677,4.0
4,-1.806857e+06,2.438869e+06,Sitio_Declarado,1,5.788832,8.124039,573.149170,1281.600586,0.000000,30.000000,15218.724609,4797.843262,41604.332031,4402.0,99.462326,0.002222,-4.395570,5.690083,5.0


### **4. Dataset Summary**

**Output:** `data/features/dataset_entrenamiento.csv`
**Next:** Notebook 04 — Training and Model Validation.